# Model comparison
This notebook evaluates a model according to an agreed upon metric in order to compare results with the GP models.

In [1]:
import numpy as np
import pandas as pds
import matplotlib.pyplot as plt

In [2]:
"""
Some loss functions
"""
def basic_mae(preds, labels):
    acc = 0 
    n = len(preds)
    for i in range(n):
        e = abs(preds[i] - labels[i])
        acc += e
 
    acc /= n
    return [acc]

def mape(preds, labels):
    acc = 0
    n = len(preds)
    for i in range(n):
        acc+=abs((labels[i] - preds[i])/labels[i])
    
    res = acc/n
    return [res]

Pick the name for your model

In [3]:
model_name = 'ANN_M3_downsampled'

In [4]:
data = pds.read_pickle(model_name + '.pkl')
data.head()

,seg,journey,speed,pred,label
0,1,12,0.199897,50.963905,55.0
1,1,12,0.250902,49.409420,46.0
2,1,12,0.289026,38.020760,37.0
3,1,12,0.329212,37.518921,36.0
4,1,12,0.371458,36.415882,35.0


In [5]:
jour_frames = []

for ij, jour in data.groupby('journey'):
    seg_frames = []
    
    for iss, seg in jour.groupby('seg'):
        tot = len(seg) - 1
        errs = {}
        i_20 = np.int_(np.round(tot * 0.2))
        i_40 = np.int_(np.round(tot * 0.4))
        i_60 = np.int_(np.round(tot * 0.6))
        i_80 = np.int_(np.round(tot * 0.8))
        
        p_20 = seg.iloc[0:i_20]
        p_40 = seg.iloc[0:i_40]
        p_60 = seg.iloc[0:i_60]
        p_80 = seg.iloc[0:i_80]
        errs['journey'] = [ij]
        errs['segment'] = [iss]
        
        errs['e_20'] = basic_mae(p_20.pred.values, p_20.label.values)
        errs['ep_20'] = mape(p_20.pred.values, p_20.label.values)
        
        errs['e_40'] = basic_mae(p_40.pred.values, p_40.label.values)
        errs['ep_40'] = mape(p_40.pred.values, p_40.label.values)
        
        errs['e_60'] = basic_mae(p_60.pred.values, p_60.label.values)
        errs['ep_60'] = mape(p_60.pred.values, p_60.label.values)
        
        errs['e_80'] = basic_mae(p_80.pred.values, p_80.label.values)
        errs['ep_80'] = mape(p_80.pred.values, p_80.label.values)
        
        seg_frames.append(pds.DataFrame(errs))
    
    jour_frames.append(pds.concat(seg_frames))
        

In [6]:
ers = pds.concat(jour_frames).reset_index(drop=True)

In [7]:
ers.head(30)

,journey,segment,e_20,ep_20,e_40,ep_40,e_60,ep_60,e_80,ep_80
0,12,1,2.229694,0.056583,1.620955,0.046715,1.487289,0.050391,1.185527,0.043605
1,12,2,1.520353,0.050833,1.434921,0.053680,1.311703,0.057597,1.078389,0.050302
2,12,3,3.031588,0.074111,3.934436,0.116122,3.387456,0.112026,2.930783,0.111437
3,12,4,2.632280,0.127439,2.243295,0.123560,2.114857,0.134544,1.881817,0.150047
4,12,5,1.766737,0.042628,1.276033,0.032783,1.321163,0.043403,1.191806,0.045828
5,12,6,1.847862,0.043116,1.226208,0.030813,0.898521,0.024178,0.813589,0.028962
6,12,7,2.648239,0.049676,1.710269,0.034200,1.383417,0.030048,1.265259,0.036920
7,12,8,0.958913,0.035601,0.801478,0.033711,0.644581,0.029297,0.543923,0.028425
8,12,9,2.891510,0.025136,4.186950,0.041242,3.485800,0.038126,3.166255,0.044341
9,12,10,1.761511,0.042860,1.721168,0.050446,1.403087,0.045072,1.244228,0.050569


In [8]:
ers.mean(axis=0)

journey    1818.397222
segment       6.000000
e_20          5.156651
ep_20         0.101104
e_40          4.290304
ep_40         0.095538
e_60          3.719077
ep_60         0.096045
e_80          3.213808
ep_80         0.099954
dtype: float64

In [9]:
ers[ers.segment != 1].mean(axis=0)

journey    1818.397222
segment       6.500000
e_20          5.251491
ep_20         0.101943
e_40          4.389618
ep_40         0.096211
e_60          3.810700
ep_60         0.096770
e_80          3.297638
ep_80         0.101094
dtype: float64

In [10]:
ers.to_pickle(model_name + '_203_errors.pkl')